In [ ]:
# https://www.spotrac.com/mls/transactions/trade/
# 
# article > table > tbody > tr:child(1) > h3 -- date - need to figure out year (could use url year )
# article > table > tbody > ts:child(2) > td > span.tradelabel -- team (remove "acquires")
# article > table > tbody > ts:child(2) > td > span.tradeplayer -- asset received (can be multiple of these)
# 
# 
# 
# result: JSON of trades
# range: 2020 to 2022

In [3]:
import requests
from bs4 import BeautifulSoup

seasons = range(2020, 2023)





b'<!doctype html >\n<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en"> <![endif]-->\n<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en"> <![endif]-->\n<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en"> <![endif]-->\n<!--[if IE 9]>    <html class="no-js ie9 oldie" lang="en"> <![endif]-->\n<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->\n<head>\n    <!-- start:global -->\n    <meta charset="utf-8">\n    <!-- end:global -->\n\n\n<!-- start:page title -->\n<title>MLS Transactions | Spotrac</title>\n<!-- end:page title -->\n\n<!-- start:meta info -->\n<meta name="keywords" content="MLS signings, transactions, news, trades, restructures, releases" />\n<meta name="description" content="Sortable NFL transactions including signings, trades, fines, suspensions, releases, etc...">\n<!-- end:meta info -->\n\n<!-- end:page title -->\n<meta property="og:title" content="MLS Transactions" />\n<meta property="og:site_name" content="Spotrac.com" />\n<meta

In [62]:
composite_list = {}

for year in seasons:
    print(f"getting spotrac transactions for {year}")
    response = requests.get(f'https://www.spotrac.com/mls/transactions/{year}/trade/')
    html = response.text
    parsed_html = BeautifulSoup(html, 'html.parser')

    result = []
    for transaction in parsed_html.body.find_all('article'):
        base_table = transaction.table
        if (base_table == None):
            continue

        rows = base_table.find_all('tr')
        base = {'year': year, 'date' : rows[0].td.h3.text, 'transactions': []}
        for club in rows[1].find_all('td'):
            club_name = club.find('span', attrs={"class":"tradelabel"}).text.replace(' acquires', '')
            raw_assets_acquired = club.find_all('span', attrs={"class":"tradeplayer"})
            assets_acquired = []
            for asset in raw_assets_acquired:
                assets_acquired.append(asset.text)
            base['transactions'].append({ 'club': club_name, 'assets_acquired': assets_acquired })

        result.append(base)
    print(f"parsed out {len(result)} items")
    composite_list[year] = result    
    
composite_list

getting spotrac transactions for 2020
parsed out 37 items
getting spotrac transactions for 2021
parsed out 40 items
getting spotrac transactions for 2022
parsed out 30 items


{2020: [{'year': 2020,
   'date': 'Dec 29',
   'transactions': [{'club': 'New York City FC',
     'assets_acquired': ['$250k 2021 General Allocation Money',
      '$250k 2022 General Allocation Money',
      '$100k additional General Allocation Money pending performance-based incentives']},
    {'club': 'Cincinnati', 'assets_acquired': ['Ronald Matarrita (D)']}]},
  {'year': 2020,
   'date': 'Dec 23',
   'transactions': [{'club': 'New York',
     'assets_acquired': ['$50k General Allocation Money',
      '2021 3rd round pick ']},
    {'club': 'Inter Miami', 'assets_acquired': ['Patrick Seagrist (D)']}]},
  {'year': 2020,
   'date': 'Dec 18',
   'transactions': [{'club': 'Chicago',
     'assets_acquired': ['2022 4th round pick (swap)']},
    {'club': 'Charlotte',
     'assets_acquired': ['Brandt Bronico (M)',
      '2022 4th round pick (swap)']}]},
  {'year': 2020,
   'date': 'Dec 17',
   'transactions': [{'club': 'New York City FC',
     'assets_acquired': ['$500k 2021 General Allocati

In [66]:
import json

with open("./mls_trades.json", "w") as outfile:
    outfile.write(json.dumps(composite_list, indent = 4))

In [68]:
with open('./mls_trades.json', 'r') as openfile:
    # Reading from json file
    corrected_file = json.load(openfile)
    
corrected_file

{'2020': [{'year': 2020,
   'date': 'Dec 29',
   'transactions': [{'club': 'New York City FC',
     'assets_acquired': ['$250000 2021 GAM',
      '$250000 2022 GAM',
      '$100000 2022 GAM (performance bonus)']},
    {'club': 'FC Cincinnati', 'assets_acquired': ['Ronald Matarrita']}]},
  {'year': 2020,
   'date': 'Dec 23',
   'transactions': [{'club': 'New York Red Bulls',
     'assets_acquired': ['$50000 2020 GAM', '2021 3rd round pick']},
    {'club': 'Inter Miami', 'assets_acquired': ['Patrick Seagrist']}]},
  {'year': 2020,
   'date': 'Dec 18',
   'transactions': [{'club': 'Chicago',
     'assets_acquired': ['2022 4th round pick (swap)']},
    {'club': 'Charlotte FC',
     'assets_acquired': ['Brandt Bronico', '2022 4th round pick (swap)']}]},
  {'year': 2020,
   'date': 'Dec 17',
   'transactions': [{'club': 'New York City FC',
     'assets_acquired': ['$500000 2021 GAM',
      '$250000 2022 GAM',
      '$500000 2022 GAM (performance bonus)']},
    {'club': 'Austin FC', 'assets_a

In [74]:
records = []
for arr in corrected_file.values():
    records += arr
records

[{'year': 2020,
  'date': 'Dec 29',
  'transactions': [{'club': 'New York City FC',
    'assets_acquired': ['$250000 2021 GAM',
     '$250000 2022 GAM',
     '$100000 2022 GAM (performance bonus)']},
   {'club': 'FC Cincinnati', 'assets_acquired': ['Ronald Matarrita']}]},
 {'year': 2020,
  'date': 'Dec 23',
  'transactions': [{'club': 'New York Red Bulls',
    'assets_acquired': ['$50000 2020 GAM', '2021 3rd round pick']},
   {'club': 'Inter Miami', 'assets_acquired': ['Patrick Seagrist']}]},
 {'year': 2020,
  'date': 'Dec 18',
  'transactions': [{'club': 'Chicago',
    'assets_acquired': ['2022 4th round pick (swap)']},
   {'club': 'Charlotte FC',
    'assets_acquired': ['Brandt Bronico', '2022 4th round pick (swap)']}]},
 {'year': 2020,
  'date': 'Dec 17',
  'transactions': [{'club': 'New York City FC',
    'assets_acquired': ['$500000 2021 GAM',
     '$250000 2022 GAM',
     '$500000 2022 GAM (performance bonus)']},
   {'club': 'Austin FC', 'assets_acquired': ['Alexander Ring']}]},


In [103]:
base_df = pd.DataFrame(records)
base_df['trade_id'] = range(0, len(base_df))
base_df['raw_trade_date'] = base_df.agg('{0[date]}, {0[year]}'.format, axis=1)
base_df['trade_date'] = pd.to_datetime(base_df['raw_trade_date'], errors='coerce')
base_df = base_df[['trade_id', 'trade_date', 'transactions']]
base_df

,trade_id,trade_date,transactions
0,0,2020-12-29,"[{'club': 'New York City FC', 'assets_acquired..."
1,1,2020-12-23,"[{'club': 'New York Red Bulls', 'assets_acquir..."
2,2,2020-12-18,"[{'club': 'Chicago', 'assets_acquired': ['2022..."
3,3,2020-12-17,"[{'club': 'New York City FC', 'assets_acquired..."
4,4,2020-12-17,"[{'club': 'Houston', 'assets_acquired': ['Fafa..."
...,...,...,...
103,103,2022-01-10,"[{'club': 'Inter Miami', 'assets_acquired': ['..."
104,104,2022-01-10,"[{'club': 'Toronto FC', 'assets_acquired': ['$..."
105,105,2022-01-06,"[{'club': 'Seattle Sounders', 'assets_acquired..."
106,106,2022-01-05,"[{'club': 'Columbus Crew', 'assets_acquired': ..."


In [104]:
transaction_expl = json.loads(base_df.explode('transactions').to_json(orient="records"))
transaction_expl_flat = pd.json_normalize(transaction_expl)

transaction_expl_flat

,trade_id,trade_date,transactions.club,transactions.assets_acquired
0,0,1609200000000,New York City FC,"[$250000 2021 GAM, $250000 2022 GAM, $100000 2..."
1,0,1609200000000,FC Cincinnati,[Ronald Matarrita]
2,1,1608681600000,New York Red Bulls,"[$50000 2020 GAM, 2021 3rd round pick]"
3,1,1608681600000,Inter Miami,[Patrick Seagrist]
4,2,1608249600000,Chicago,[2022 4th round pick (swap)]
...,...,...,...,...
212,105,1641427200000,Charlotte FC,[Chris Hegardt]
213,106,1641340800000,Columbus Crew,"[$300000 2022 GAM, $150000 2022 GAM (performan..."
214,106,1641340800000,Colorado Rapids,[Aboubacar Keita]
215,107,1641254400000,LAFC,"[$100000 2022 GAM, $75000 2022 GAM (performanc..."


In [110]:
compiled_assets = pd.DataFrame({ 'trade_id' : [], 'transactions.club' : [], 'assets': []})

for i, row in transaction_expl_flat.iterrows():
    new_assets = []
    raw_assets = row['transactions.assets_acquired']
    
    for asset in raw_assets:
        base_asset_record = {'asset_type': None, 'value' : None}
        if ('GAM' in asset):
            base_asset_record['asset_type'] = 'GAM'
        elif (' pick' in asset):
            base_asset_record['asset_type'] = 'Draft Pick'
        elif ('International' in asset):
            base_asset_record['asset_type'] = 'International Slot'
        elif ('sell-on' in asset or 'sellon' in asset or 'sell on' in asset):
            base_asset_record['asset_type'] = 'Sell-On Clause'
        else:
            base_asset_record['asset_type'] = 'Player'
        
        base_asset_record['value'] = asset
        new_assets.append(base_asset_record)
        
    compiled_assets = compiled_assets.append({ 'trade_id': row.trade_id, 'transactions.club': row['transactions.club'], 'assets': new_assets }, ignore_index=True)

compiled_assets  

,trade_id,transactions.club,assets
0,0.0,New York City FC,"[{'asset_type': 'GAM', 'value': '$250000 2021 ..."
1,0.0,FC Cincinnati,"[{'asset_type': 'Player', 'value': 'Ronald Mat..."
2,1.0,New York Red Bulls,"[{'asset_type': 'GAM', 'value': '$50000 2020 G..."
3,1.0,Inter Miami,"[{'asset_type': 'Player', 'value': 'Patrick Se..."
4,2.0,Chicago,"[{'asset_type': 'Draft Pick', 'value': '2022 4..."
...,...,...,...
212,105.0,Charlotte FC,"[{'asset_type': 'Player', 'value': 'Chris Hega..."
213,106.0,Columbus Crew,"[{'asset_type': 'GAM', 'value': '$300000 2022 ..."
214,106.0,Colorado Rapids,"[{'asset_type': 'Player', 'value': 'Aboubacar ..."
215,107.0,LAFC,"[{'asset_type': 'GAM', 'value': '$100000 2022 ..."


In [111]:
composite_transactions = pd.merge(transaction_expl_flat, compiled_assets, left_on=['trade_id','transactions.club'], right_on=['trade_id','transactions.club'])
composite_transactions

,trade_id,trade_date,transactions.club,transactions.assets_acquired,assets
0,0,1609200000000,New York City FC,"[$250000 2021 GAM, $250000 2022 GAM, $100000 2...","[{'asset_type': 'GAM', 'value': '$250000 2021 ..."
1,0,1609200000000,FC Cincinnati,[Ronald Matarrita],"[{'asset_type': 'Player', 'value': 'Ronald Mat..."
2,1,1608681600000,New York Red Bulls,"[$50000 2020 GAM, 2021 3rd round pick]","[{'asset_type': 'GAM', 'value': '$50000 2020 G..."
3,1,1608681600000,Inter Miami,[Patrick Seagrist],"[{'asset_type': 'Player', 'value': 'Patrick Se..."
4,2,1608249600000,Chicago,[2022 4th round pick (swap)],"[{'asset_type': 'Draft Pick', 'value': '2022 4..."
...,...,...,...,...,...
212,105,1641427200000,Charlotte FC,[Chris Hegardt],"[{'asset_type': 'Player', 'value': 'Chris Hega..."
213,106,1641340800000,Columbus Crew,"[$300000 2022 GAM, $150000 2022 GAM (performan...","[{'asset_type': 'GAM', 'value': '$300000 2022 ..."
214,106,1641340800000,Colorado Rapids,[Aboubacar Keita],"[{'asset_type': 'Player', 'value': 'Aboubacar ..."
215,107,1641254400000,LAFC,"[$100000 2022 GAM, $75000 2022 GAM (performanc...","[{'asset_type': 'GAM', 'value': '$100000 2022 ..."


In [162]:
asset_expl = json.loads(composite_transactions.explode('assets').to_json(orient="records"))
asset_expl_flat = pd.json_normalize(asset_expl)
asset_expl_flat = asset_expl_flat[['trade_id', 'trade_date', 'transactions.club', 'assets.asset_type', 'assets.value']]
asset_expl_flat.columns = ['trade_id', 'trade_date', 'club','acquired_asset_type', 'acquired_asset_value']
asset_expl_flat.trade_date = pd.to_datetime(asset_expl_flat.trade_date, unit='ms')
asset_expl_flat['acquired_asset_id'] = range(0, len(asset_expl_flat))
asset_expl_flat

,trade_id,trade_date,club,acquired_asset_type,acquired_asset_value,acquired_asset_id
0,0,2020-12-29,New York City FC,GAM,$250000 2021 GAM,0
1,0,2020-12-29,New York City FC,GAM,$250000 2022 GAM,1
2,0,2020-12-29,New York City FC,GAM,$100000 2022 GAM (performance bonus),2
3,0,2020-12-29,FC Cincinnati,Player,Ronald Matarrita,3
4,1,2020-12-23,New York Red Bulls,GAM,$50000 2020 GAM,4
...,...,...,...,...,...,...
289,106,2022-01-05,Columbus Crew,GAM,$150000 2022 GAM (performance bonus),289
290,106,2022-01-05,Colorado Rapids,Player,Aboubacar Keita,290
291,107,2022-01-04,LAFC,GAM,$100000 2022 GAM,291
292,107,2022-01-04,LAFC,GAM,$75000 2022 GAM (performance bonus),292


In [179]:
import numpy as np 

asset_expl_flat['gam_value'] = asset_expl_flat.acquired_asset_value.str.extract('(\d+) \d{4} GAM')
asset_expl_flat['gam_value'] = np.where(asset_expl_flat.gam_value.isna(), 0, asset_expl_flat.gam_value)
asset_expl_flat['asset_year'] = asset_expl_flat.acquired_asset_value.str.extract('\d+ (\d{4}) GAM')
asset_expl_flat['asset_year'] = np.where(asset_expl_flat.asset_year.isna(), asset_expl_flat.trade_date.dt.year, asset_expl_flat.asset_year)
asset_expl_flat.asset_year = asset_expl_flat.asset_year.astype(int)
asset_expl_flat.gam_value = asset_expl_flat.gam_value.astype(int)
asset_expl_flat

,trade_id,trade_date,club,acquired_asset_type,acquired_asset_value,acquired_asset_id,gam_value,asset_year
0,0,2020-12-29,New York City FC,GAM,$250000 2021 GAM,0,250000,2021
1,0,2020-12-29,New York City FC,GAM,$250000 2022 GAM,1,250000,2022
2,0,2020-12-29,New York City FC,GAM,$100000 2022 GAM (performance bonus),2,100000,2022
3,0,2020-12-29,FC Cincinnati,Player,Ronald Matarrita,3,0,2020
4,1,2020-12-23,New York Red Bulls,GAM,$50000 2020 GAM,4,50000,2020
...,...,...,...,...,...,...,...,...
289,106,2022-01-05,Columbus Crew,GAM,$150000 2022 GAM (performance bonus),289,150000,2022
290,106,2022-01-05,Colorado Rapids,Player,Aboubacar Keita,290,0,2022
291,107,2022-01-04,LAFC,GAM,$100000 2022 GAM,291,100000,2022
292,107,2022-01-04,LAFC,GAM,$75000 2022 GAM (performance bonus),292,75000,2022


In [175]:
# save state
asset_expl_flat.to_csv('./mls_trade_db.csv', index=False)

In [176]:
asset_expl_flat.club.unique()

array(['New York City FC', 'FC Cincinnati', 'New York Red Bulls',
       'Inter Miami', 'Chicago', 'Charlotte FC', 'Austin FC', 'Houston',
       'FC Dallas', 'CF Montreal', 'Portland', 'LAFC', 'Atlanta United',
       'San Jose', 'Columbus Crew', 'Vancouver Whitecaps', 'Minnesota',
       'Seattle Sounders', 'Nashville', 'D.C. United', 'Colorado Rapids',
       'New England', 'Philadelphia', 'Kansas City', 'LA Galaxy',
       'Real Salt Lake', 'Orlando City', 'Toronto FC'], dtype=object)

In [182]:
# create team GAM table for a year
# group each trade's rows
# for each trade
# - add sum of GAM values for acquired teams (IE: their rows) to team table
# - subtract for non-team rows still in the trade subtract GAM value from team table
asset_expl_flat

,trade_id,trade_date,club,acquired_asset_type,acquired_asset_value,acquired_asset_id,gam_value,asset_year
0,0,2020-12-29,New York City FC,GAM,$250000 2021 GAM,0,250000,2021
1,0,2020-12-29,New York City FC,GAM,$250000 2022 GAM,1,250000,2022
2,0,2020-12-29,New York City FC,GAM,$100000 2022 GAM (performance bonus),2,100000,2022
3,0,2020-12-29,FC Cincinnati,Player,Ronald Matarrita,3,0,2020
4,1,2020-12-23,New York Red Bulls,GAM,$50000 2020 GAM,4,50000,2020
...,...,...,...,...,...,...,...,...
289,106,2022-01-05,Columbus Crew,GAM,$150000 2022 GAM (performance bonus),289,150000,2022
290,106,2022-01-05,Colorado Rapids,Player,Aboubacar Keita,290,0,2022
291,107,2022-01-04,LAFC,GAM,$100000 2022 GAM,291,100000,2022
292,107,2022-01-04,LAFC,GAM,$75000 2022 GAM (performance bonus),292,75000,2022


In [184]:
gam_table = {  }

available_asset_years = range(int(asset_expl_flat.asset_year.min()), int(asset_expl_flat.asset_year.max()) + 1)
for year in available_asset_years:
    gam_table[str(year)] = {}
    for club in asset_expl_flat.club.unique():
        gam_table[str(year)][club] = 0
    
# there's a three team trade in here to look out for...
grouped_trades = asset_expl_flat.groupby('trade_id')
for trade_id, trade in grouped_trades:
    gam_groups = trade.groupby('club')

    if (len(gam_groups) == 2):
        for club, rows in gam_groups:
            for year in available_asset_years:
                gam_add = rows[rows.asset_year == year].gam_value.sum()
                used_assets = rows.acquired_asset_id.tolist()
                gam_remove = trade[(trade.asset_year == year) & ~(trade.acquired_asset_id.isin(used_assets))].gam_value.sum()
                
                selected_asset_year = str(year)
                print(f'{selected_asset_year}: Adding {gam_add} to {club}')
                gam_table[selected_asset_year][club] += gam_add
                print(f'{selected_asset_year}: Subtracting {gam_remove} from {club}')
                gam_table[selected_asset_year][club] -= gam_remove
    else:
        print(f'trade_id {trade_id} had weird number of teams: {len(gam_groups)}')
        
gam_table  

2020: Adding 0 to FC Cincinnati
2020: Subtracting 0 from FC Cincinnati
2021: Adding 0 to FC Cincinnati
2021: Subtracting 250000 from FC Cincinnati
2022: Adding 0 to FC Cincinnati
2022: Subtracting 350000 from FC Cincinnati
2023: Adding 0 to FC Cincinnati
2023: Subtracting 0 from FC Cincinnati
2024: Adding 0 to FC Cincinnati
2024: Subtracting 0 from FC Cincinnati
2025: Adding 0 to FC Cincinnati
2025: Subtracting 0 from FC Cincinnati
2020: Adding 0 to New York City FC
2020: Subtracting 0 from New York City FC
2021: Adding 250000 to New York City FC
2021: Subtracting 0 from New York City FC
2022: Adding 350000 to New York City FC
2022: Subtracting 0 from New York City FC
2023: Adding 0 to New York City FC
2023: Subtracting 0 from New York City FC
2024: Adding 0 to New York City FC
2024: Subtracting 0 from New York City FC
2025: Adding 0 to New York City FC
2025: Subtracting 0 from New York City FC
2020: Adding 0 to Inter Miami
2020: Subtracting 50000 from Inter Miami
2021: Adding 0 to Int

2022: Adding 0 to Vancouver Whitecaps
2022: Subtracting 0 from Vancouver Whitecaps
2023: Adding 0 to Vancouver Whitecaps
2023: Subtracting 0 from Vancouver Whitecaps
2024: Adding 0 to Vancouver Whitecaps
2024: Subtracting 0 from Vancouver Whitecaps
2025: Adding 0 to Vancouver Whitecaps
2025: Subtracting 0 from Vancouver Whitecaps
2020: Adding 0 to Colorado Rapids
2020: Subtracting 0 from Colorado Rapids
2021: Adding 150000 to Colorado Rapids
2021: Subtracting 0 from Colorado Rapids
2022: Adding 0 to Colorado Rapids
2022: Subtracting 0 from Colorado Rapids
2023: Adding 0 to Colorado Rapids
2023: Subtracting 0 from Colorado Rapids
2024: Adding 0 to Colorado Rapids
2024: Subtracting 0 from Colorado Rapids
2025: Adding 0 to Colorado Rapids
2025: Subtracting 0 from Colorado Rapids
2020: Adding 0 to Minnesota
2020: Subtracting 0 from Minnesota
2021: Adding 0 to Minnesota
2021: Subtracting 150000 from Minnesota
2022: Adding 0 to Minnesota
2022: Subtracting 0 from Minnesota
2023: Adding 0 to M

2020: Adding 0 to CF Montreal
2020: Subtracting 0 from CF Montreal
2021: Adding 0 to CF Montreal
2021: Subtracting 1000000 from CF Montreal
2022: Adding 0 to CF Montreal
2022: Subtracting 0 from CF Montreal
2023: Adding 0 to CF Montreal
2023: Subtracting 0 from CF Montreal
2024: Adding 0 to CF Montreal
2024: Subtracting 0 from CF Montreal
2025: Adding 0 to CF Montreal
2025: Subtracting 0 from CF Montreal
2020: Adding 0 to Nashville
2020: Subtracting 0 from Nashville
2021: Adding 1000000 to Nashville
2021: Subtracting 0 from Nashville
2022: Adding 0 to Nashville
2022: Subtracting 0 from Nashville
2023: Adding 0 to Nashville
2023: Subtracting 0 from Nashville
2024: Adding 0 to Nashville
2024: Subtracting 0 from Nashville
2025: Adding 0 to Nashville
2025: Subtracting 0 from Nashville
2020: Adding 0 to Inter Miami
2020: Subtracting 0 from Inter Miami
2021: Adding 0 to Inter Miami
2021: Subtracting 0 from Inter Miami
2022: Adding 0 to Inter Miami
2022: Subtracting 0 from Inter Miami
2023: A

2024: Adding 0 to San Jose
2024: Subtracting 0 from San Jose
2025: Adding 0 to San Jose
2025: Subtracting 0 from San Jose
2020: Adding 0 to FC Cincinnati
2020: Subtracting 0 from FC Cincinnati
2021: Adding 0 to FC Cincinnati
2021: Subtracting 50000 from FC Cincinnati
2022: Adding 0 to FC Cincinnati
2022: Subtracting 75000 from FC Cincinnati
2023: Adding 0 to FC Cincinnati
2023: Subtracting 0 from FC Cincinnati
2024: Adding 0 to FC Cincinnati
2024: Subtracting 0 from FC Cincinnati
2025: Adding 0 to FC Cincinnati
2025: Subtracting 0 from FC Cincinnati
2020: Adding 0 to New York Red Bulls
2020: Subtracting 0 from New York Red Bulls
2021: Adding 50000 to New York Red Bulls
2021: Subtracting 0 from New York Red Bulls
2022: Adding 75000 to New York Red Bulls
2022: Subtracting 0 from New York Red Bulls
2023: Adding 0 to New York Red Bulls
2023: Subtracting 0 from New York Red Bulls
2024: Adding 0 to New York Red Bulls
2024: Subtracting 0 from New York Red Bulls
2025: Adding 0 to New York Red 

2020: Adding 0 to New York Red Bulls
2020: Subtracting 0 from New York Red Bulls
2021: Adding 450000 to New York Red Bulls
2021: Subtracting 0 from New York Red Bulls
2022: Adding 0 to New York Red Bulls
2022: Subtracting 0 from New York Red Bulls
2023: Adding 0 to New York Red Bulls
2023: Subtracting 0 from New York Red Bulls
2024: Adding 0 to New York Red Bulls
2024: Subtracting 0 from New York Red Bulls
2025: Adding 0 to New York Red Bulls
2025: Subtracting 0 from New York Red Bulls
2020: Adding 0 to CF Montreal
2020: Subtracting 0 from CF Montreal
2021: Adding 0 to CF Montreal
2021: Subtracting 0 from CF Montreal
2022: Adding 0 to CF Montreal
2022: Subtracting 0 from CF Montreal
2023: Adding 0 to CF Montreal
2023: Subtracting 0 from CF Montreal
2024: Adding 0 to CF Montreal
2024: Subtracting 0 from CF Montreal
2025: Adding 0 to CF Montreal
2025: Subtracting 0 from CF Montreal
2020: Adding 0 to Houston
2020: Subtracting 0 from Houston
2021: Adding 0 to Houston
2021: Subtracting 0 fr

2022: Subtracting 425000 from FC Cincinnati
2023: Adding 0 to FC Cincinnati
2023: Subtracting 0 from FC Cincinnati
2024: Adding 0 to FC Cincinnati
2024: Subtracting 0 from FC Cincinnati
2025: Adding 0 to FC Cincinnati
2025: Subtracting 0 from FC Cincinnati
2020: Adding 0 to FC Cincinnati
2020: Subtracting 0 from FC Cincinnati
2021: Adding 0 to FC Cincinnati
2021: Subtracting 0 from FC Cincinnati
2022: Adding 0 to FC Cincinnati
2022: Subtracting 75000 from FC Cincinnati
2023: Adding 0 to FC Cincinnati
2023: Subtracting 0 from FC Cincinnati
2024: Adding 0 to FC Cincinnati
2024: Subtracting 0 from FC Cincinnati
2025: Adding 0 to FC Cincinnati
2025: Subtracting 0 from FC Cincinnati
2020: Adding 0 to New York City FC
2020: Subtracting 0 from New York City FC
2021: Adding 0 to New York City FC
2021: Subtracting 0 from New York City FC
2022: Adding 75000 to New York City FC
2022: Subtracting 0 from New York City FC
2023: Adding 0 to New York City FC
2023: Subtracting 0 from New York City FC
2

2024: Adding 0 to Colorado Rapids
2024: Subtracting 0 from Colorado Rapids
2025: Adding 0 to Colorado Rapids
2025: Subtracting 0 from Colorado Rapids
2020: Adding 0 to LAFC
2020: Subtracting 0 from LAFC
2021: Adding 0 to LAFC
2021: Subtracting 0 from LAFC
2022: Adding 0 to LAFC
2022: Subtracting 1500000 from LAFC
2023: Adding 0 to LAFC
2023: Subtracting 0 from LAFC
2024: Adding 0 to LAFC
2024: Subtracting 0 from LAFC
2025: Adding 0 to LAFC
2025: Subtracting 0 from LAFC
2020: Adding 0 to Charlotte FC
2020: Subtracting 0 from Charlotte FC
2021: Adding 0 to Charlotte FC
2021: Subtracting 0 from Charlotte FC
2022: Adding 0 to Charlotte FC
2022: Subtracting 0 from Charlotte FC
2023: Adding 0 to Charlotte FC
2023: Subtracting 0 from Charlotte FC
2024: Adding 0 to Charlotte FC
2024: Subtracting 0 from Charlotte FC
2025: Adding 0 to Charlotte FC
2025: Subtracting 0 from Charlotte FC
2020: Adding 0 to Inter Miami
2020: Subtracting 0 from Inter Miami
2021: Adding 0 to Inter Miami
2021: Subtracti

{'2020': {'New York City FC': 0,
  'FC Cincinnati': 0,
  'New York Red Bulls': 50000,
  'Inter Miami': 100000,
  'Chicago': 0,
  'Charlotte FC': 0,
  'Austin FC': -850000,
  'Houston': -350000,
  'FC Dallas': 325000,
  'CF Montreal': -750000,
  'Portland': 550000,
  'LAFC': 525000,
  'Atlanta United': 875000,
  'San Jose': 575000,
  'Columbus Crew': -200000,
  'Vancouver Whitecaps': 125000,
  'Minnesota': 300000,
  'Seattle Sounders': -250000,
  'Nashville': -325000,
  'D.C. United': -650000,
  'Colorado Rapids': -75000,
  'New England': -25000,
  'Philadelphia': 50000,
  'Kansas City': 0,
  'LA Galaxy': 0,
  'Real Salt Lake': 0,
  'Orlando City': 0,
  'Toronto FC': 0},
 '2021': {'New York City FC': 950000,
  'FC Cincinnati': 775000,
  'New York Red Bulls': -1400000,
  'Inter Miami': 1425000,
  'Chicago': 400000,
  'Charlotte FC': 725000,
  'Austin FC': -500000,
  'Houston': -743675,
  'FC Dallas': 0,
  'CF Montreal': -1050000,
  'Portland': 1167000,
  'LAFC': 493675,
  'Atlanta United